# Data Prep

The goal of this notebook is to prep the data for the alignment task.  This includes computing audio features and generating a query list.

In [2]:
import numpy as np
import librosa as lb
import os
import os.path
from pathlib import Path
import multiprocessing

In [3]:
ANNOTATIONS_ROOT = Path('Chopin_Mazurkas/annotations_beat')
AUDIO_ROOT = Path('Chopin_Mazurkas/wav_22050_mono')
FEATURES_ROOT = Path('features')
train_files = Path('cfg_files/filelist.train.txt')
test_files = Path('cfg_files/filelist.test.txt')

In [17]:
if not os.path.exists(FEATURES_ROOT):
    os.mkdir(FEATURES_ROOT)

### Create Train-toy, Train-small, Train-medium, Train-full, Test-full filelists

In [6]:
import random
from itertools import combinations

In [49]:
def create_internal_datasets(num_pairs, filelist, piece, outdir): # piece is e.g. 'Chopin_Op017No4'
    '''creates a file with num_pairs combiantions of file names in piece'''
    # load file names from original file list
    total_files = []
    with open(filelist, 'r') as infile:
        for line in infile:
            if piece in line:
                total_files.append(line)
    # generate all pairs
    all_pairs = list(combinations(total_files, 2))
    subset = random.sample(range(0, len(all_pairs)), num_pairs) # select num_pairs random pairs from all_pairs
    # write to outfile
    with open(outdir,'a') as outfile:
        for index in subset:
            item1, item2 = all_pairs[index]
            outfile.write(item1[:-1]+' '+item2)   
    return

In [50]:
def create_allpairs_datasets(filelist, piece, outdir):
    '''creates a file with all possible combinations of file names in piece'''
    total_files = []
    with open(filelist, 'r') as infile:
        for line in infile:
            if piece in line:
                total_files.append(line)
    # generate all pairs
    all_pairs = list(combinations(total_files, 2))
    # write to outfile
    with open(outdir,'a') as outfile:
        for pair in all_pairs:
            item1, item2 = pair[0], pair[1]
            outfile.write(item1[:-1]+' '+item2)   
    return

In [51]:
# GENERATE TRAIN DATASETS
# Train-toy: randomly select 5 pairs from Op 17 #4
create_internal_datasets(5, train_files, 'Chopin_Op017No4', 'cfg_files/filelist.train_toy.txt')

# Train-small: randomly select 200 pairs from Op 17 #4
create_internal_datasets(200, train_files, 'Chopin_Op017No4', 'cfg_files/filelist.train_small.txt')

# Train-medium: includes all (63 choose 2) pairs for Op 17 #4
create_internal_datasets(1953, train_files, 'Chopin_Op017No4', 'cfg_files/filelist.train_medium.txt')

# Train-full: includes all (63 choose 2) + (88, choose 2) pairs for Op 17 #4 and Op 63 #3
create_internal_datasets(1953, train_files, 'Chopin_Op017No4', 'cfg_files/filelist.train_full.txt')
create_internal_datasets(3828, train_files, 'Chopin_Op063No3', 'cfg_files/filelist.train_full.txt')

In [52]:
# GENERATE TEST DATASETS
# Test-full: includes all pairs from the 3 test Mazurkas
create_allpairs_datasets(test_files, 'Chopin_Op024No2', 'cfg_files/filelist.test_full.txt')
create_allpairs_datasets(test_files, 'Chopin_Op030No2', 'cfg_files/filelist.test_full.txt')
create_allpairs_datasets(test_files, 'Chopin_Op068No3', 'cfg_files/filelist.test_full.txt')

### Generate query list

Here we generate a file containing each pair of files to be aligned.

In [8]:
def generate_query_list(filelist, outfile): # Does what we already did before
    
    # group files by piece
    d = {}
    with open(filelist, 'r') as f:
        for line in f:
            parts = line.strip().split('/')
            assert len(parts) == 2
            piece, fileid = parts
            if piece not in d:
                d[piece] = []
            d[piece].append(fileid)
            
    # print out all pairings
    with open(outfile, 'w') as fout:
        for piece in d:
            num_recordings = len(d[piece])
            for i in range(num_recordings):
                fileid1 = d[piece][i]
                for j in range(i+1, num_recordings):
                    fileid2 = d[piece][j]
                    line = f'{piece}/{fileid1} {piece}/{fileid2}\n'
                    fout.write(line)
                    
    return

In [9]:
train_queries = 'cfg_files/query.train.list'
test_queries = 'cfg_files/query.test.list'
generate_query_list(train_files, train_queries)
generate_query_list(test_files, test_queries)

### Construct 7 modified benchmark datasets

#### Find median duration for each Mazurka

In [ ]:
def get_median(filelist, piece):
    '''get median duration of recordings in one Mazurka folder'''
    lengths = []
    with open(filelist, 'r') as cfg_file:
        # for each line, if line contains the Mazurka
        for line in cfg_file:
            if piece in line:
                #   make path to .wav file
                path_to_wav = 'Chopin_Mazurkas/wav_22050_mono/{}.wav'.format(line[:-1])
                #   calculate duration
                duration = lb.get_duration(filename=path_to_wav)
                #   append to list
                lengths.append(duration)

    lengths.sort()
    median = lengths[len(lengths)//2]
    return median

In [ ]:
Chopin_Op017No4_MEDIAN = get_median('cfg_files/filelist.train.txt','Chopin_Op017No4')
Chopin_Op063No3_MEDIAN = get_median('cfg_files/filelist.train.txt','Chopin_Op063No3')
Chopin_Op024No2_MEDIAN = get_median('cfg_files/filelist.test.txt','Chopin_Op024No2')
Chopin_Op030No2_MEDIAN = get_median('cfg_files/filelist.test.txt','Chopin_Op030No2')
Chopin_Op068No3_MEDIAN = get_median('cfg_files/filelist.test.txt','Chopin_Op068No3')

#### Create separate directories for each dataset

In [ ]:
def create_mazurka_directories(time_modify):
    '''create separate directories for each time scale modification'''
    for item in time_modify:
        dir = 'Mazurkas_median_x{}'.format(item)
        ann_dir = '{}/annotations_beat'.format(dir)
        wav_dir = '{}/wav_22050_mono'.format(dir)
        for mazurka in ['Chopin_Op017No4','Chopin_Op024No2','Chopin_Op030No2','Chopin_Op063No3','Chopin_Op068No3']:
            ann_piece_dir = '{}/{}'.format(ann_dir, mazurka)
            wav_piece_dir = '{}/{}'.format(wav_dir, mazurka)
            if not os.path.exists(ann_piece_dir):
                os.makedirs(ann_piece_dir)
            if not os.path.exists(wav_piece_dir):
                os.makedirs(wav_piece_dir)

In [ ]:
factors = ['1.000','0.500','0.630','0.794','1.260','1.588','2.000']
create_mazurka_directories(factors)

#### Perform time-scale modification

##### Generate .wav files

In [ ]:
import soundfile as sf
import IPython.display as ipd
from tsm_tools import *
import pandas as pd

In [ ]:
def time_scale_batch(filelist, piece, median_duration, factor, outdir, n_cores):
    '''time-scale modify for all .wav files for one mazurka for one factor'''
    inputs = []
    with open(filelist, 'r') as cfg_file:
        for line in cfg_file:
            if piece in line:
                # make path to .wav file
                path_to_wav = 'Chopin_Mazurkas/wav_22050_mono/{}.wav'.format(line[:-1])
                inputs.append((path_to_wav, median_duration, factor, outdir))
    
    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(time_scale_single, inputs)
    
    return

In [ ]:
def time_scale_single(path_to_wav, median_duration, factor, outdir):
    '''time-scale modify for one .wav file'''
    y, sr = lb.load(path_to_wav)
    # do time scale modification
    old_duration = lb.get_duration(y=y, sr=sr)
    ratio = (median_duration*factor) / old_duration
    y_mod = tsm_hybrid(y, alpha=ratio)
    # save to new .wav file
    new_path_to_wav = '{}{}'.format(outdir, path_to_wav[15:]) # e.g. 'Mazurkas_median_x1.000/wav_22050_mono/Chopin_Op017No4/Chopin_Op017No4_Kitain-1937_pid9163-02.wav'
    #lb.output.write_wav(new_path_to_wav, y, sr)
    sf.write(new_path_to_wav, y_mod, sr)
    return

In [ ]:
def time_scale_by_mazurka(filelist, piece, median_duration, factors):
    '''time-scale modify by all factors with one mazurka'''
    for factor in factors:
        time_scale_batch(filelist, piece, median_duration, float(factor),'Mazurkas_median_x{}'.format(factor), 24)
        print("Done with Mazurkas_median_x{}/{}".format(factor, piece))
    return

In [ ]:
time_scale_by_mazurka(train_files,'Chopin_Op017No4',Chopin_Op017No4_MEDIAN,factors)

In [ ]:
time_scale_by_mazurka(train_files,'Chopin_Op063No3',Chopin_Op063No3_MEDIAN,factors)

In [ ]:
time_scale_by_mazurka(test_files,'Chopin_Op024No2',Chopin_Op024No2_MEDIAN,factors)

In [ ]:
time_scale_by_mazurka(test_files,'Chopin_Op030No2',Chopin_Op030No2_MEDIAN,factors)

In [ ]:
time_scale_by_mazurka(test_files,'Chopin_Op068No3',Chopin_Op068No3_MEDIAN,factors)

##### Generate .beat files

In [ ]:
import pandas as pd

In [ ]:
def getTimestamps(annotfile):
    df = pd.read_csv(annotfile, header=None, sep='\s+', skiprows=3)
    return np.array(df[0])

In [ ]:
def create_beat_annotation(in_path_beat, in_path_wav, out_path, num_multiply): # num_multiply should be median*factor
    '''modifies beat annotations for one .wav file'''
    # load .wav file to get original duration
    real_duration = lb.get_duration(filename=in_path_wav)
    # (median*factor)/real_dur
    original_ts = getTimestamps(in_path_beat)
    modified_ts = (original_ts*num_multiply)/real_duration
    index = 0
    curr_read = 0
    new_lines = []
    with open(in_path_beat, 'r') as infile:
        for line in infile:
            if curr_read < 3: # copy first 3 lines
                new_lines.append(line)
                curr_read += 1
                continue
            else: # modify the rest of the lines
                label = line.split('\t')[-1]
                if index+1 == len(modified_ts):
                    curr_start, curr_end = modified_ts[index], 0.0
                else:
                    curr_start, curr_end = modified_ts[index], modified_ts[index+1]
                new_string = '{}\t{}\t{}'.format(curr_start, curr_end, label)
                new_lines.append(new_string)
                index += 1
    with open(out_path, 'w') as outfile:
        for line in new_lines:
            outfile.write(line)
    return new_lines

In [ ]:
# create_beat_annotation('Chopin_Mazurkas/annotations_beat/Chopin_Op017No4/Chopin_Op017No4_Weissenberg-1971_pid9052b-09.beat', 'Chopin_Mazurkas/wav_22050_mono/Chopin_Op017No4/Chopin_Op017No4_Weissenberg-1971_pid9052b-09.wav', 'hello.beat', Chopin_Op017No4_MEDIAN*2)

In [ ]:
def create_beat_annotation_batch(filelist, piece, median_duration, factor, outdir, n_cores): # e.g. outdir is 'Mazurkas_median_x2.000/annotations_beat/Chopin_Op017No4'
    '''modifies beat annotations for all .wav files for one Mazurka for one factor'''
    inputs = []
    with open(filelist, 'r') as cfg_file:
        for line in cfg_file:
            if piece in line:
                path_to_beat = 'Chopin_Mazurkas/annotations_beat/{}.beat'.format(line[:-1])
                path_to_wav = 'Chopin_Mazurkas/wav_22050_mono/{}.wav'.format(line[:-1])
                out_path = 'Mazurkas_median_x{}/annotations_beat/{}.beat'.format(factor, line[:-1])
                num_multiply = median_duration*float(factor)
                inputs.append((path_to_beat, path_to_wav, out_path, num_multiply))
    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(create_beat_annotation, inputs)
    
    return


In [ ]:
# create_beat_annotation_batch(train_files, 'Chopin_Op017No4', Chopin_Op017No4_MEDIAN, '2.000', 'Mazurkas_median_x2.000/annotations_beat/Chopin_Op017No4', 24)

In [ ]:
def beat_annotation_by_mazurka(filelist, piece, median_duration, factors):
    '''modifies beat annotations by all factors with one mazurka'''
    for factor in factors:
        create_beat_annotation_batch(filelist, piece, median_duration, factor, 'Mazurkas_median_x{}/annotations_beat/{}'.format(factor, piece), 24)
        print("Done with Mazurkas_median_x{}/annotations_beat/{}".format(factor, piece))
    return

In [ ]:
beat_annotation_by_mazurka(train_files, 'Chopin_Op017No4', Chopin_Op017No4_MEDIAN, factors)
beat_annotation_by_mazurka(train_files, 'Chopin_Op063No3', Chopin_Op063No3_MEDIAN, factors)
beat_annotation_by_mazurka(test_files, 'Chopin_Op024No2', Chopin_Op024No2_MEDIAN, factors)
beat_annotation_by_mazurka(test_files, 'Chopin_Op030No2', Chopin_Op030No2_MEDIAN, factors)
beat_annotation_by_mazurka(test_files, 'Chopin_Op068No3', Chopin_Op068No3_MEDIAN, factors)

### Compute features on clean audio

First we compute features on the audio.

##### Compute features on the 'Chopin_Mazurkas' directory

In [28]:
def compute_chroma_single(infile, outfile, sr = 22050, hop_length=512):
    y, sr = lb.core.load(infile, sr = sr)
    #F = lb.feature.chroma_cens(y, sr=sr, hop_length=hop_length)
    F = lb.feature.chroma_cqt(y, sr=sr, hop_length=hop_length, norm=2)
    np.save(outfile, F)
    return

In [8]:
def compute_chroma_batch(filelist, outdir, n_cores):
    
    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (AUDIO_ROOT / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [ ]:
FEATS_CLEAN_DIR = FEATURES_ROOT / 'clean'
compute_chroma_batch(train_files, FEATS_CLEAN_DIR, 24)
compute_chroma_batch(test_files, FEATS_CLEAN_DIR, 24)

##### Compute features on the modified median directories e.g. 'Mazurkas_median_x0.500'

In [37]:
def compute_chroma_median(filelist, median_dir, outdir, n_cores): # outdir should be 'features/clean'
    '''compute features for each median directory e.g. for all .wav files in 'Mazurkas_median_x0.500' '''
    
    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = median_dir / outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (median_dir / Path('wav_22050_mono') / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [30]:
FEATS_CLEAN_DIR = FEATURES_ROOT / 'clean'
median_directories = ['Mazurkas_median_x0.500', 'Mazurkas_median_x0.630', 'Mazurkas_median_x0.794', 'Mazurkas_median_x1.000', 'Mazurkas_median_x1.260', 'Mazurkas_median_x1.588', 'Mazurkas_median_x2.000']

In [ ]:
# compute chromas for each directory
for median_dir in median_directories:
    compute_chroma_median(train_files, median_dir, FEATS_CLEAN_DIR, 24)
    compute_chroma_median(test_files, median_dir, FEATS_CLEAN_DIR, 24)
    print("Finished generating features for {}".format(median_dir))